# Introduction to Model Context Provider (MCP) Architecture

In the previous notebook, we created a restaurant booking agent using the Strands Agents SDK with tools directly defined within our application. In this notebook, we'll evolve our architecture to use the Model Context Provider (MCP) pattern.

## What is MCP?

Model Context Provider (MCP) is a protocol that allows AI assistants to access external tools, data, and services through a standardized interface. The MCP architecture separates the concerns of:

1. **Tool Implementation**: Specialized functionality exposed through an MCP server
2. **Agent Orchestration**: The AI agent that uses these tools to accomplish tasks

This separation offers several benefits:

- **Modularity**: Tools can be developed and maintained independently from agents
- **Reusability**: MCP servers can be used by multiple different agents and applications
- **Scalability**: Tool collections can grow without modifying agent code
- **Cross-Application Support**: Tools can be used across various AI applications supporting the MCP protocol

## Our Implementation

We've implemented our restaurant booking tools in a separate MCP server that exposes them via the MCP protocol. The agent connects to this server to use these tools. This architecture allows us to:

1. Run the tools in a separate process
2. Potentially reuse the tools across multiple agents
3. Update the tools independently from the agent

![Agent Architecture](../images/agent-with-mcp.png)

In our updated implementation:

- **restaurant_mcp_server.py**: Contains the tool implementations wrapped in an MCP server using FastMCP


Let's explore this architecture!

In [ ]:
import os
import json
import socket
import subprocess
import time
from IPython.display import display, HTML
import pandas as pd
import uuid
import boto3
from typing import Dict, Any, Optional, List
import pandas as pd
from IPython.display import Markdown, display
from utils import create_dynamodb, selectAllFromDynamodb

In [ ]:
# Setup DynamoDB table for restaurant bookings
dynamodb = boto3.resource('dynamodb')
table_name = 'restaurant_bookings'
create_dynamodb(table_name)  # Create the table if it doesn't exist
table = dynamodb.Table(table_name)

## 1. How We Wrapped Tools in the MCP Server

Let's take a look at how we implemented the MCP server. We used FastMCP, which makes it easy to wrap functions as tools and expose them via the MCP protocol.

In [ ]:
# Display part of the MCP server implementation
with open('restaurant_mcp_server.py', 'r') as f:
    code = f.read()

display(Markdown(f"```{code}```"))

As you can see, wrapping a function as an MCP tool is simple with FastMCP:

1. Create a FastMCP instance
2. Decorate functions with `@mcp.tool()`
3. Run the server with `mcp.run()`

The MCP server automatically generates the schema for each tool based on the function signatures and docstrings.

## 2. Create MCP Client

In [ ]:
from strands.tools.mcp import MCPClient
from mcp import StdioServerParameters, stdio_client

session = boto3.Session()
credentials = session.get_credentials()
creds = credentials.get_frozen_credentials()

env = os.environ.copy()
env["AWS_ACCESS_KEY_ID"] = creds.access_key
env["AWS_SECRET_ACCESS_KEY"] = creds.secret_key
env["AWS_SESSION_TOKEN"] = creds.token
env["AWS_DEFAULT_REGION"] = "us-west-2"

restaurant_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="python", 
            args=["./restaurant_mcp_server.py", "--table-name", table_name], 
            env=env
        )
    )
)

## 3. Set up the agent

In [ ]:
# Define the agent's system prompt (instructions)
from strands import Agent, tool
from strands.models import BedrockModel

agent_instruction="""
## Role
You are a ABC Restaurant Booking agent. You are in charge of restaurant reservations.

## Instructions
- Handle restaurant reservations inquiries and requests from users
- Create new bookings when requested with appropriate details
- Retrieve booking information when asked
- Cancel reservations when requested
- Be professional and courteous in all interactions

## Output Requirements
- When responding to the end user, don't output your thinking steps
- Only give useful information to the end user
- Confirm all successful bookings, changes, and cancellations clearly
"""

# Initialize the Amazon Bedrock model
model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",  # Using Amazon Nova Pro model
    max_tokens=3000,
    temperature=1,
    top_p=1,
    additional_request_fields={
        "inferenceConfig": {
            "topK": 1,
        },
    }
)

## 4. Test the agent under the MCP client context

In [ ]:
with restaurant_mcp_client:
    tools = restaurant_mcp_client.list_tools_sync()

    # Create the Strands Agent with our defined tools
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    response = agent(
        """Hi, my name is Jane Doe.
        I want to book a table for 2 tomorrow at 5pm.
        """
    )
    print(response)

In [ ]:
items = selectAllFromDynamodb(table_name)
items

In [ ]:
with restaurant_mcp_client:
    tools = restaurant_mcp_client.list_tools_sync()

    # Create the Strands Agent with our defined tools
    agent = Agent(
        model=model,
        system_prompt=agent_instruction,
        tools=[tools],
        callback_handler=None
    )

    response = agent(
        """Is there a reserversation made under the name Jane Doe?"""
    )
    print(response)

# Conclusion

In this notebook, we've explored how to use the Model Context Provider (MCP) architecture to build a restaurant booking agent. This approach demonstrates a more modular and scalable way to build AI agents, where tools are exposed through a standardized protocol.

The FastMCP library makes it easy to wrap functions as tools and expose them via the MCP protocol. This allows us to focus on implementing the actual tool functionality without worrying about the details of the protocol.

The MCP architecture is particularly valuable for complex applications where:
- Multiple agents need to share tools
- Tools need to be maintained independently from agents
- New capabilities need to be added without changing agent code
- Tools need to be reused across different AI applications